1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию

In [4]:
# скачаем catboost
!pip install catboost

     |████████████████████████████████| 69.2MB 45kB/s 


In [59]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import catboost as cb

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import precision_recall_curve, roc_auc_score
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

import matplotlib.pyplot as plt

In [1]:
# загрузим датасет
from google.colab import files
uploaded = files.upload()

Saving train_case2.csv to train_case2.csv


In [7]:
df = pd.read_csv('train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [12]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop(['id','cardio'], 1), 
                                                    df['cardio'], random_state=0)

In [13]:
X_train.shape

(52500, 11)

К полям:

gender, cholesterol применим OHE-кодирование
age, height, weight, ap_hi, ap_lo - standardScaler
gluc, smoke, alco, active - оставим пока как есть

In [14]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

Теперь объединим все наши трансформеры с помощью FeatureUnion

In [15]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

Стоим модель на основе логистической регрессии

In [34]:
classifier_lr = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(C = 0.50,random_state = 42)),
])

#запустим кросс-валидацию
cv_scores = cross_val_score(classifier_lr, X_train, y_train, cv=6, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

#обучим пайплайн на всем тренировочном датасете
classifier_lr.fit(X_train, y_train)
y_score = classifier_lr.predict_proba(X_test)[:, 1]

CV score is 0.7864164472716206+-0.004926372379000047


найдем лучший порог и посчитаем метрики

In [35]:
b=1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

metrics_lr = np.array([precision[ix], recall[ix],fscore[ix], roc_auc_score(y_test.values, y_score)]) 

Best Threshold=0.387930, F-Score=0.730, Precision=0.648, Recall=0.837


Стоим модель на основе случайного леса

In [39]:
classifier_rf = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(max_depth=3, min_samples_leaf=20, random_state=42)),
])

#запустим кросс-валидацию
cv_scores = cross_val_score(classifier_rf, X_train, y_train, cv=6, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

#обучим пайплайн на всем тренировочном датасете
classifier_rf.fit(X_train, y_train)
y_score = classifier_rf.predict_proba(X_test)[:, 1]

CV score is 0.7879598850085493+-0.0036292737154484936


найдем лучший порог и посчитаем метрики

In [48]:
b=1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall+0.0000001)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))
metrics_rf = np.array([precision[ix], recall[ix],fscore[ix], roc_auc_score(y_test.values, y_score)]) 

Best Threshold=0.420106, F-Score=0.736, Precision=0.679, Recall=0.804


Стоим модель на основе CatBoostClassifier

In [49]:
classifier_cb = Pipeline([
    ('features',feats),
    ('classifier', cb.CatBoostClassifier(max_depth=3, random_state=42, verbose=False, )),
])

#запустим кросс-валидацию
cv_scores = cross_val_score(classifier_cb, X_train, y_train, cv=6, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

#обучим пайплайн на всем тренировочном датасете
classifier_cb.fit(X_train, y_train)
y_score = classifier_cb.predict_proba(X_test)[:, 1]

CV score is 0.8026035713708897+-0.004449524887128347


найдем лучший порог и посчитаем метрики

In [50]:
b=1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))
metrics_cb = np.array([precision[ix], recall[ix],fscore[ix], roc_auc_score(y_test.values, y_score)]) 

Best Threshold=0.391759, F-Score=0.741, Precision=0.696, Recall=0.792


3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)

In [55]:
models_list = ['LogReg', 'RandFor', 'CatbClassifier' ]
comp_table = pd.DataFrame(np.array([metrics_lr, metrics_rf, metrics_cb]),
                          columns=['pr', 'rec', 'auc', 'f_score'], index=models_list)

comp_table                                                                                                         

,pr,rec,auc,f_score
LogReg,0.647877,0.836866,0.730344,0.783810
RandFor,0.678693,0.804032,0.736065,0.788980
CatbClassifier,0.696070,0.791820,0.740865,0.803228


4. сделать выводы о том, какая модель справилась с задачей лучше других

**С точки зрения интегральных оценок (ROC-AUC и F-score) лучше всех справился CatboostClassifier**

5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого).
p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). Допустим, у нас две модели:

первая помечает 100 объектов как класс 1, но TP = 90
вторая помечает 1000 объектов как класс 1, но TP такой же - 90

**Очевидно, что лучшей метрикой при сильном дисбалансе классов является precision_recall_curve , т.к. эта метрика сфокусирована на меньшем классе, в то время как roc_auc_curve зависит от обеих классов. Кроме того, одной из осей roc_auc_curve является FPR = FP / (FP + TN). В первом из описанных случаев он будет равен 1/9990, а во втором случае 91/9990. Т.е. в обеих случаях очень низкое значение и графики roc_auc_curve будут неотличимы на глаз, хотя по сути разница этих моделей огромна.**